In [1]:
import os
import sys
import argparse 
import math
import time
import h5py
import joblib
import subprocess
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
os.chdir('/Users/pengl7/Downloads/WGS/compare-variants/NIST/')

In [3]:
%ls -lth

total 114176
-rw-r--r--  1 pengl7  NIH\Domain Users    22M Sep 28 11:51 long_cleared_QD.csv
-rw-r--r--  1 pengl7  NIH\Domain Users    21M Sep 25 16:58 long_cleared.csv
-rw-r-----  1 pengl7  NIH\Domain Users    12M Sep 25 10:35 long4_with_title


In [6]:
# df = pd.read_csv("long4_with_title", sep="\t",index_col="POS", na_values={}, dtype={'UNMATCH': "category", "DP": "float", "GQ": "float", "MQ": "float"}, )
# the file of long_cleared_QD has been cleared on the previous notebook
df = pd.read_csv("long_cleared_QD.csv")

In [7]:
print(df.shape)
print(df.columns.to_list())

(339850, 10)
['POS', 'UNMATCH', 'TYPE', 'QUAL', 'FS', 'AF', 'DP', 'GQ', 'MQ', 'QD']


In [8]:
df.dtypes

POS          int64
UNMATCH      int64
TYPE         int64
QUAL       float64
FS         float64
AF         float64
DP         float64
GQ         float64
MQ         float64
QD         float64
dtype: object

In [12]:
df.describe()

,POS,UNMATCH,TYPE,QUAL,FS,AF,DP,GQ,MQ,QD
count,3.398500e+05,339850.000000,339850.000000,339850.000000,339850.000000,339850.000000,339850.000000,339850.000000,339850.000000,339850.000000
mean,1.256498e+08,0.102430,0.161136,480.453901,1.734002,0.701234,19.791552,78.838855,59.430577,24.165223
std,7.641346e+07,0.303214,0.367657,720.776478,3.342272,0.247944,22.881223,27.072391,2.707447,12.603838
min,1.014600e+04,0.000000,0.000000,30.250000,0.000000,0.000200,1.000000,0.000000,20.000000,0.119897
25%,5.925464e+07,0.000000,0.000000,233.770000,0.000000,0.500000,14.000000,59.000000,60.000000,13.952692
50%,1.148287e+08,0.000000,0.000000,385.770000,0.000000,0.500000,19.000000,99.000000,60.000000,20.125909
75%,1.950169e+08,0.000000,0.000000,647.770000,2.218000,1.000000,24.000000,99.000000,60.000000,37.985625
max,2.489456e+08,1.000000,1.000000,117461.000000,357.189000,1.000000,3142.000000,99.000000,60.000000,1006.255000


## Apply logsistic regression model

In [17]:
this_formula = "UNMATCH ~ TYPE + QD + DP + GQ + QUAL + FS + AF + MQ"
res = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit() 
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               339850
Model:                            GLM   Df Residuals:                   339841
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -85927.
Date:                Mon, 28 Sep 2020   Deviance:                   1.7185e+05
Time:                        13:13:30   Pearson chi2:                 1.14e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.9782      0.123    114.023      0.000      13.738      14.219
TYPE           1.7102      0.015    117.920      0.000       1.682       1.739
QD            -0.0181      0.001    -19.784      0.000      -0.020      -0.016
DP             0.0070      0.001      8.729      0.000       0.005       0.009
GQ            -0.0256      0.000    -83.832      0.000      -0.026      -0.025
QUAL          -0.0001    2.9e-05     -3.790      0.000      -0.000   -5.31e-05
FS             0.0100      0.002      4.743      0.000       0.006       0.014
AF            -1.5972      0.046    -34.435      0.000      -1.688      -1.506
MQ            -0.2259      0.002   -114.321      0.000      -0.230      -0.222
==============================================================================
"""

In [18]:
print("Coefficeients")
print(res.params)
print()
print("p-Values")
print(res.pvalues)
print()
print("Dependent variables")
print(res.model.endog_names)

Coefficeients
Intercept    13.978229
TYPE          1.710170
QD           -0.018136
DP            0.006970
GQ           -0.025610
QUAL         -0.000110
FS            0.009991
AF           -1.597204
MQ           -0.225902
dtype: float64

p-Values
Intercept     0.000000e+00
TYPE          0.000000e+00
QD            4.070075e-87
DP            2.567716e-18
GQ            0.000000e+00
QUAL          1.508077e-04
FS            2.107636e-06
AF           7.493848e-260
MQ            0.000000e+00
dtype: float64

Dependent variables
UNMATCH


In [19]:
# if set TYPE as categorical
df["TYPE"] = df["TYPE"].astype("category")

In [20]:
this_formula = "UNMATCH ~ TYPE + QD + DP + GQ + QUAL + FS + AF + MQ"
res3 = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit() 
res3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               339850
Model:                            GLM   Df Residuals:                   339841
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -85927.
Date:                Mon, 28 Sep 2020   Deviance:                   1.7185e+05
Time:                        13:21:12   Pearson chi2:                 1.14e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.9782      0.123    114.023      0.000      13.738      14.219
TYPE[T.1]      1.7102      0.015    117.920      0.000       1.682       1.739
QD            -0.0181      0.001    -19.784      0.000      -0.020      -0.016
DP             0.0070      0.001      8.729      0.000       0.005       0.009
GQ            -0.0256      0.000    -83.832      0.000      -0.026      -0.025
QUAL          -0.0001    2.9e-05     -3.790      0.000      -0.000   -5.31e-05
FS             0.0100      0.002      4.743      0.000       0.006       0.014
AF            -1.5972      0.046    -34.435      0.000      -1.688      -1.506
MQ            -0.2259      0.002   -114.321      0.000      -0.230      -0.222
==============================================================================
"""

In [21]:
print("Coefficeients")
print(res3.params)
print()
print("p-Values")
print(res3.pvalues)
print()
print("Dependent variables")
print(res3.model.endog_names)

Coefficeients
Intercept    13.978229
TYPE[T.1]     1.710170
QD           -0.018136
DP            0.006970
GQ           -0.025610
QUAL         -0.000110
FS            0.009991
AF           -1.597204
MQ           -0.225902
dtype: float64

p-Values
Intercept     0.000000e+00
TYPE[T.1]     0.000000e+00
QD            4.070075e-87
DP            2.567716e-18
GQ            0.000000e+00
QUAL          1.508077e-04
FS            2.107636e-06
AF           7.493848e-260
MQ            0.000000e+00
dtype: float64

Dependent variables
UNMATCH


In [15]:

formula2 = "UNMATCH ~ POS + TYPE + QD + DP + GQ + QUAL + FS + AF + MQ"
res2 = sm.formula.glm(formula=formula2, family=sm.families.Binomial(), data=df).fit() 
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               339850
Model:                            GLM   Df Residuals:                   339840
Model Family:                Binomial   Df Model:                            9
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -85811.
Date:                Mon, 28 Sep 2020   Deviance:                   1.7162e+05
Time:                        13:09:35   Pearson chi2:                 1.04e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.0760      0.123    114.516      0.000      13.835      14.317
POS        -1.286e-09   8.48e-11    -15.161      0.000   -1.45e-09   -1.12e-09
TYPE           1.7152      0.015    118.190      0.000       1.687       1.744
QD            -0.0179      0.001    -19.611      0.000      -0.020      -0.016
DP             0.0069      0.001      8.697      0.000       0.005       0.008
GQ            -0.0256      0.000    -83.638      0.000      -0.026      -0.025
QUAL          -0.0001   2.88e-05     -3.771      0.000      -0.000   -5.22e-05
FS             0.0100      0.002      4.735      0.000       0.006       0.014
AF            -1.5984      0.046    -34.445      0.000      -1.689      -1.507
MQ            -0.2250      0.002   -113.805      0.000      -0.229      -0.221
==============================================================================
"""